### Importing Libraries

In [28]:
import pandas as pd
import numpy as np
from textblob import TextBlob 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
import re
import nltk
nltk.download('punkt')
import textstat
from textstat.textstat import textstatistics,legacy_round

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kedar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Reading the csv

In [2]:
df = pd.read_csv("Cleaned.csv")
df.head()

,Url_Title,Content
0,How is Login Logout Time Tracking for Employee...,When people hear AI they often think about sen...
1,How does AI help to monitor Retail Shelf watch...,"With increasing computing power and more data,..."
2,AI and its impact on the Fashion Industry - Bl...,If you were a fan of the 90s film Clueless bac...
3,How do deep learning models predict old and ne...,"Understanding exactly how data is ingested, an..."
4,How artificial intelligence can boost your pro...,"From the stone age to the modern world, from h..."


In [3]:
df['Content']=df['Content'].str.replace(',','')

In [4]:
stopWordsFile =r'C:\Users\Kedar\Desktop\20211030 Test Assignment/StopWords_Generic.txt'
positiveWordsFile =r'C:\Users\Kedar\Desktop\20211030 Test Assignment/PositiveWords.txt'
nagitiveWordsFile =r'C:\Users\Kedar\Desktop\20211030 Test Assignment/NegativeWords.txt'

In [5]:
with open(stopWordsFile ,'r') as stop_words:
    stopWords = stop_words.read().lower()
    stopWordList = stopWords.split('\n')
    stopWordList[-1:] = []

In [6]:
def tokenizer(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    filtered_words = list(filter(lambda token: token not in stopWordList, tokens))
    return filtered_words

In [7]:
#Loading positive words
with open(positiveWordsFile,'r') as posfile:
    positivewords=posfile.read().lower()
    positiveWordList=positivewords.split('\n')

In [8]:
#Loading negative words
with open(nagitiveWordsFile ,'r') as negfile:
    negativeword=negfile.read().lower()
    negativeWordList=negativeword.split('\n')

In [12]:
def positive_score(text):
    PosWords = 0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in positiveWordList:
            PosWords  += 1
    
    sumPos = PosWords
    return sumPos

In [13]:
def negative_score(text):
    NegWords=0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in negativeWordList:
            NegWords -=1
    sumNeg = NegWords 
    sumNeg = sumNeg * -1
    return sumNeg

In [14]:
df["positive_score"] = df["Content"].apply(positive_score)
df["negative_score"] = df["Content"].apply(negative_score)

In [15]:
#Calculating polarity
polarity = []

for i in df["Content"]:
    polarity.append(TextBlob(i).sentiment.polarity)

In [17]:
df["polarity_score"] = polarity

In [18]:
#Calculating subjectivity
subjectivity = []

for i in df["Content"]:
    subjectivity.append(TextBlob(i).sentiment.subjectivity)

In [19]:
df["subjectivity_score"] = subjectivity

In [20]:
# Calculating Average sentence length 
     
def average_sentence_length(text):
    sentence_list = sent_tokenize(text)
    tokens = tokenizer(text)
    totalWordCount = len(tokens)
    totalSentences = len(sentence_list)
    average_sent = 0
    if totalSentences != 0:
        average_sent = totalWordCount / totalSentences
    
    average_sent_length= average_sent
    
    return round(average_sent_length)

In [21]:
df["Average sentence length"] = df["Content"].apply(average_sentence_length)

In [23]:
#Calculating Percentage complex words
def percentage_complex_word(text):
    tokens = tokenizer(text)
    complexWord = 0
    complex_word_percentage = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    if len(tokens) != 0:
        complex_word_percentage = complexWord/len(tokens)
    
    return complex_word_percentage

In [24]:
df["Percentage complex words"] = df["Content"].apply(percentage_complex_word)

In [25]:
df["Percentage complex words"] = df["Percentage complex words"].astype(float)

In [26]:
#Calculating Fog Index
def fog_index(average_sentence_length, percentage_complex_word):
    fogIndex = 0.4 * (average_sentence_length + percentage_complex_word)
    return fogIndex

In [29]:
df["Fog index"] = np.vectorize(fog_index)(df['Average sentence length'],df['Percentage complex words'])

In [34]:
#Calculating Avg word per sentence
def average_wordper_sent(text):
    sentence_list = sent_tokenize(text)
    tokens = tokenizer(text)
    totalWord = len(tokens)
    totalSent = len(sentence_list)
    if totalSent != 0:
        average_word_sent = totalWord / totalSent
    
    return round(average_word_sent)

In [35]:
df["Avg words per sentence"] = df["Content"].apply(average_wordper_sent)

In [36]:
##Calculating  complex word count
def complex_word_count(text):
    tokens = tokenizer(text)
    complexWord = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    return complexWord

In [37]:
df["Complex word count"] = df["Content"].apply(complex_word_count)

In [38]:
#Counting total words
def word_count(text):
    tokens = tokenizer(text)
    return len(tokens)

In [39]:
df["Word Count"] = df["Content"].apply(word_count)

In [49]:
#Calculating Syallable per word
def syllables_per_word(text):
    syllable = syllables_count(text)
    words = word_count(text)
    SPW = float(syllable) / float(words)
    return legacy_round(SPW, 1)

In [50]:
df["Syllable per word"] = df["Content"].apply(syllables_per_word)

In [54]:
# Calculating Personal Pronouns 
def pronouns(text):
    count =0
    pro = ['I','we','my','ours','us','We','My','Ours','Us']
    for i in text:
        if i in pro:
            count = count +1
    return count

In [55]:
df["Personal pronoun"] = df["Content"].apply(pronouns)

In [62]:
#Calculating Average word length
def avg_word_len(text):
    char_list = textstat.char_count(text, ignore_spaces=True)
    tokens = tokenizer(text)
    totalWord = len(tokens)
    totalchar = (char_list)
    if totalWord!= 0:
        avg_char = totalchar / totalWord
    
    return round(avg_char)

In [63]:
df["Average word length"] = df['Content'].apply(avg_word_len)

### Importing the Input.csv

In [66]:
df1 = pd.read_csv("Input.csv")
df1.head()

,URL_ID,URL
0,1.0,https://insights.blackcoffer.com/how-is-login-...
1,2.0,https://insights.blackcoffer.com/how-does-ai-h...
2,3.0,https://insights.blackcoffer.com/ai-and-its-im...
3,4.0,https://insights.blackcoffer.com/how-do-deep-l...
4,5.0,https://insights.blackcoffer.com/how-artificia...


### Creating new df for final CSV

In [68]:
df3 = pd.concat([df1,df],axis = 1)
df3.head(3)

,URL_ID,URL,Url_Title,Content,positive_score,negative_score,polarity_score,subjectivity_score,Average sentence length,Percentage complex words,Fog index,Avg words per sentence,Complex word count,Word Count,Syllable per word,Personal pronoun,Average word length
0,1.0,https://insights.blackcoffer.com/how-is-login-...,How is Login Logout Time Tracking for Employee...,When people hear AI they often think about sen...,4,6,0.143809,0.485273,19,0.314815,7.725926,19,136,432,2.6,15,8
1,2.0,https://insights.blackcoffer.com/how-does-ai-h...,How does AI help to monitor Retail Shelf watch...,With increasing computing power and more data ...,9,6,0.177208,0.493776,15,0.376590,6.150636,15,148,393,2.7,11,8
2,3.0,https://insights.blackcoffer.com/ai-and-its-im...,AI and its impact on the Fashion Industry - Bl...,If you were a fan of the 90s film Clueless bac...,35,23,0.141999,0.542371,15,0.391652,6.156661,15,441,1126,2.6,51,8


In [69]:
del df3["Url_Title"]

In [70]:
del df3["Content"]

In [71]:
df3.head()

,URL_ID,URL,positive_score,negative_score,polarity_score,subjectivity_score,Average sentence length,Percentage complex words,Fog index,Avg words per sentence,Complex word count,Word Count,Syllable per word,Personal pronoun,Average word length
0,1.0,https://insights.blackcoffer.com/how-is-login-...,4,6,0.143809,0.485273,19,0.314815,7.725926,19,136,432,2.6,15,8
1,2.0,https://insights.blackcoffer.com/how-does-ai-h...,9,6,0.177208,0.493776,15,0.376590,6.150636,15,148,393,2.7,11,8
2,3.0,https://insights.blackcoffer.com/ai-and-its-im...,35,23,0.141999,0.542371,15,0.391652,6.156661,15,441,1126,2.6,51,8
3,4.0,https://insights.blackcoffer.com/how-do-deep-l...,5,1,0.077056,0.458329,18,0.441860,7.376744,18,114,258,2.8,2,9
4,5.0,https://insights.blackcoffer.com/how-artificia...,10,11,0.089617,0.492307,11,0.428928,4.571571,11,172,401,2.8,28,9


In [72]:
df3.to_csv('TextAnalysisOutput.csv',index = None)